# Beamline 8.3.2 GPU Reconstruction Example

### Setup

In [5]:
import numpy as np
import astra
vol = np.ones((100,120,120))
theta = np.linspace(0,np.pi,180)
vol_geom = astra.create_vol_geom(vol.shape[1],vol.shape[2],vol.shape[0])
proj_geom = astra.create_proj_geom('parallel3d', 1.0, 1.0, vol.shape[1], vol.shape[2], theta)
proj_id, tomo = astra.create_sino3d_gpu(vol, proj_geom, vol_geom)

### This is the python-wrapped function that should be run on multiple GPUs

In [2]:
def astra_recon(tomo,theta,vol):
    vol_geom = astra.create_vol_geom(vol.shape)
    proj_geom = astra.create_proj_geom('parallel', 1.0, vol.shape[0], theta)
    proj_id = astra.create_projector('cuda', proj_geom, vol_geom)
    rec_id = astra.data2d.create('-vol', vol_geom)
    sino_id = astra.data2d.create('-sino', proj_geom, tomo)
    cfg = astra.astra_dict('FBP_CUDA')
    cfg['ReconstructionDataId'] = rec_id
    cfg['ProjectionDataId'] = sino_id
    cfg['ProjectorId'] = proj_id

    alg_id = astra.algorithm.create(cfg)
    astra.algorithm.run(alg_id)

    recon = astra.data2d.get(rec_id)
    return recon
    

### This for loop should be split among multiple GPUs

In [3]:
recon = np.zeros_like(vol)
for i, (sino, img) in enumerate(zip(tomo,vol)):
    recon[i] = astra_recon(sino,theta,img)

### Recon should be same size as original vol

In [4]:
print(f"vol: {vol.shape}")
print(f"recon: {recon.shape}")

vol: (100, 120, 120)
recon: (100, 120, 120)
